<a href="https://colab.research.google.com/github/olaviinha/SloppyButcher/blob/master/chopper_core_early-poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chops up a Google Drive directory full of audio files into short clips, shuffles them and concatenates them according to BPM and clipsPerBeat. **Note that the output file will have the same length as your input files combined.**

HOWTO:

1.   Place audio files into a directory in your Google Drive.
2.   Type the path of that directory in the `input_dir` field.
3.   Type the desired path and filename of the output file (to be created) in the `output_file` field.
4.   Set desired `bpm` and `clipsPerBeat`
5.   Hit Runtime -> Run all
6.   Audio players with preview clips will appear in the Preview section when done. Depending on the number and lengths of your input files, it may take a minute.




In [ ]:
#@title Mount Google Drive
#@markdown Root of your Google Drive is `/content/drive/My Drive/`
from google.colab import drive
drive.mount('/content/drive')

### Configuration

In [ ]:
input_dir = "/content/drive/My Drive/tmp" #@param {type:"string"}
output_file = "/content/drive/My Drive/incoming/test1.wav" #@param {type:"string"}
vol = 0.75 #@param {type:"slider", min:0, max:1, step:0.05}
bpm = 120 #@param {type:"slider", min:100, max:200, step:1}
clipsPerBeat = 2 #@param {type:"slider", min:1, max:8, step:1}


### Rituals & magic

In [ ]:
!gsutil -q -m cp -R gs://olaviinha/slicer/* /content/tmp
%cd {input_dir}

import ntpath, random, os, math, librosa
from os import listdir
from os.path import isfile, join
from glob import glob
from IPython.display import Audio
import numpy as np

vol = 0.8
maxArgs = 3500
min = 60
minutekb = 16479.4921875
allfiles = []
allConvFiles = []
allChopFiles = []

def strList(s):
    str1 = " -v " + str(vol) + " " 
    return (str1.join(s)) 

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def process(filelist, outputDir, type):
  i=0
  for file in filelist:  
    input = file
    if type is "convert":
      output = outputDir+"co"+str(i)+".wav"
      !ffmpeg -hide_banner -loglevel panic -y -i "{input}" -c:a pcm_s24le -ar 48000 -af "pan=stereo|c0=c0|c1=c0" "{output}"
    if type is "trim":
      output = outputDir+"tr"+str(i)+".wav"
      !sox -S "{input}" {output} trim 0 {trim_dur}
    i+=1
  
def chop(inputDir, outputDir, filelist, slice):
  i = 0
  for file in filelist:
    print(file)
    input = str(inputDir)+str(file)
    output = str(outputDir)+"ch"+str(i)+".wav"
    !sox -S {input} {output} trim 0 {slice} : newfile : restart
    i+=1

def removeResidue(filelist):
  basesize = os.path.getsize(dir_chops+"0001.wav")
  for file in filelist:
    if os.path.getsize(file) < basesize:
      !rm {file}


In [ ]:
allfiles = []
for ext in ('*.wav', '*.aiff', '*.flac', '*.mp3', '*.ogg'):
   allfiles.extend(glob(join(input_dir, ext)))
i = 0
for file in allfiles:
  input = str(file)
  name = path_leaf(str(file).split(".")[0])
  output = "/content/tmp/conv/n"+str(i)+".wav"
  !ffmpeg -hide_banner -loglevel panic -i "{input}" -c:a pcm_s24le -ar 48000 -af "pan=stereo|c0=c0|c1=c0" "{output}"
  i+=1

In [ ]:
allConvFiles = [f for f in listdir("/content/tmp/conv") if isfile(join("/content/tmp/conv", f))]
i = 0
for file in allConvFiles:
  n = i
  input = "/content/tmp/conv/"+str(file)
  output = "/content/tmp/chop/"+str(n)+".wav"
  !sox -S "{input}" {output} trim 0 {slice} : newfile : restart
  i+=1

In [ ]:
basesize = os.path.getsize('/content/tmp/chop/0001.wav')
allChops = glob("/content/tmp/chop/*.wav")
for chop in allChops:
   if os.path.getsize(chop) < basesize:
     !rm {chop}
     print('Removed', chop)

In [ ]:
allChops = glob("/content/tmp/chop/*.wav")
random.shuffle(allChops)
maxArgs = 3700

if len(allChops) > maxArgs:
  !mkdir "/content/tmp/parts"
  arDivider = math.ceil(len(allChops)/maxArgs)
  npAllChops = np.array(allChops)
  chopPart = np.array_split(npAllChops, arDivider)

  i=0;
  for part in chopPart:
    filelist = "-v " + str(vol) + " " + strList(part.tolist())
    partOutFile = "/content/tmp/parts/pt"+str(i)+".wav"
    !sox -S {filelist} -r 48000 -c 2 -b 24 {partOutFile}
    i+=1

  allParts = glob("/content/tmp/parts/*.wav")
  filelist = "-v " + str(vol) + " " + strList(allParts)
else:
  filelist = "-v " + str(vol) + " " + strList(allChops)

print("Input: "+filelist)
!sox -S {filelist} -r 48000 -c 2 -b 24 "{output_file}"
print("\n>> File saved to "+output_file)

In [ ]:
!ffmpeg -hide_banner -loglevel panic -y -i "{output_file}" -b:a 320k -ss 0 -t 120 /content/preview.mp3
!ffmpeg -hide_banner -loglevel panic -y -i "{output_file}" -ss 0 -t 120 /content/preview.wav
baselen = 60/bpm
kicklen1 = baselen*0.75
kicklen2 = baselen*1.25
!sox /content/tmp/kick-heavy.wav /content/tmp/trimmed-heavy.wav trim 0 {baselen}
!sox /content/tmp/kick-punchy.wav /content/tmp/trimmed-punchy1.wav trim 0 {kicklen1}
!sox /content/tmp/kick-punchy.wav /content/tmp/trimmed-punchy2.wav trim 0 {kicklen2}
!sox /content/tmp/trimmed-punchy1.wav /content/tmp/trimmed-punchy2.wav /content/tmp/punchy-block.wav
repeatCount = bpm*2
!sox /content/tmp/trimmed-heavy.wav /content/tmp/loop-heavy.wav repeat {repeatCount}
!sox /content/tmp/punchy-block.wav /content/tmp/loop-punchy.wav repeat {bpm}
!sox -m -v 0.6 /content/tmp/loop-heavy.wav "/content/preview.wav" /content/tmp/out-heavy-mix.wav
!sox -m /content/tmp/loop-punchy.wav "/content/preview.wav" /content/tmp/out-punchy-mix.wav
!ffmpeg -hide_banner -loglevel panic -y -i /content/tmp/out-heavy-mix.wav -b:a 320k -ss 0 -t 120 /content/preview-heavy.mp3
!ffmpeg -hide_banner -loglevel panic -y -i /content/tmp/out-punchy-mix.wav -b:a 320k -ss 0 -t 120 /content/preview-punchy.mp3

### Previews

In [ ]:
# Preview clip from the file saved in your Drive
Audio("/content/preview.mp3")

In [ ]:
# Preview clip with additional beat
Audio("/content/preview-punchy.mp3")

In [ ]:
# Preview clip with additional beat
Audio("/content/preview-heavy.mp3")